In [56]:
import boto3
import os
import json
from pprint import pprint
from dotenv import load_dotenv
import pandas as pd

In [17]:
# 認証情報を環境変数から読み込み
load_dotenv()

Python-dotenv could not parse statement starting at line 7


True

In [18]:
# 認証情報を直接指定
session = boto3.Session(
    aws_access_key_id=os.getenv('aws_access_key_id'),
    aws_secret_access_key=os.getenv('aws_secret_access_key'),
    region_name=os.getenv('region_name')
)

### 2つのナレッジベースにアクセスする準備を実施

In [19]:
region_name = "us-east-1"
# ナレッジベースID chunk 100分割の方
knowledge_base_id1 = "EBUXJ8RLCT"
# ナレッジベースID デフォルト 300分割の方
knowledge_base_id2 = "YFB3JIZ2VQ"
# ナレッジベースID ローカル編集　データ投入用
knowledge_base_id3 = "HF5C3OQ1EL"

kb_client_runtime = session.client("bedrock-agent-runtime", region_name=region_name)
bedrock_runtime = session.client(service_name="bedrock-runtime", region_name=region_name)

# 推論用モデル(今回はClaudeのv2を使います)
model_id = "anthropic.claude-v2"
# クエリ拡張用のモデル(Claude Instantを使います。これは簡単なタスクにおける速度向上が狙いです)
q_model_id = "anthropic.claude-instant-v1"

In [20]:
# 推論
# 推論用コード
def invoke_claude(text, model_id, max_tokens_to_sample=1000):
    body = json.dumps({
        "prompt": f"\n\nHuman:{text}\n\nAssistant: ",
        "max_tokens_to_sample": max_tokens_to_sample,
        "temperature": 0.1,
        "top_p": 0.9,
    })
    accept = "application/json"
    content_type = "application/json"

    response = bedrock_runtime.invoke_model(
        body=body,
        modelId=model_id,
        accept=accept,
        contentType=content_type
    )

    response_body = json.loads(response.get('body').read())
    return response_body.get('completion')[1:]

# クエリ拡張用コード
def generate_queries(original_query, n=4):
    f = ''
    for i in range(n):
        f += f'{n}: \n'

    prompt = f'''
以下に示すQueryはユーザの入力した検索クエリです。
このクエリに関連するクエリを多角的な視点から{n}つ生成してください
Formatに従って結果を出力してください

<Query>
{original_query}
</Query>

<Format>
{f}
</Format>
'''
    result = invoke_claude(prompt, q_model_id)
    result = result.split('<Format>')[1].split('</Format>')[0]
    # print(result)
    generated_queries = []
    for q in result.split('\n'):
        print("拡張クエリ", q)
        if q == '':
            continue
        generated_queries.append(q.split(' ')[1])
    # ここでなぜか解答を真ん中2つに絞っている？　確かに解答前後は空白ではあるが？
    return generated_queries[1:-1]

In [21]:
# RAG検索用コード
def kb_search(query, knowledge_base_id, n=5):
    res = kb_client_runtime.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=knowledge_base_id,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': n
            }
        }
    )

    # {doc: score}の形に整形します
    return_dict = {}
    for r in res['retrievalResults']:
        return_dict[r['content']['text']] = r['score']

    return return_dict

In [22]:
# RRFのコード
def reciprocal_rank_fusion(all_results, k=50):
    fused_scores = {}
    for query, doc_scores in all_results.items():
        for rank, (doc, score) in enumerate(sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)):
            if doc not in fused_scores:
                fused_scores[doc] = 0
            previous_score = fused_scores[doc]
            fused_scores[doc] += 1 / (rank + k)

    reranked_results = {doc: score for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)}
    return reranked_results

In [23]:
# 問い合わせprompt
original_guery = """
経営者による財政状態、経営成績及びキャッシュ・フローの状況分析について教えてください
財政状態及び経営成績の状態について、アームがNasdaq Grobal Select Marketへ上場したと思うのですが、そちらについて詳しい説明をしてください
"""

queries = generate_queries(original_guery)
queries.insert(0, original_guery)
print(f"queryの一覧：{queries}")


拡張クエリ 
拡張クエリ 4: 財務諸表の分析方法
拡張クエリ 4: 企業価値の評価
拡張クエリ 4: 上場企業の財務諸表開示
拡張クエリ 4: 株式市場への上場手続き
拡張クエリ 
拡張クエリ 
queryの一覧：['\n経営者による財政状態、経営成績及びキャッシュ・フローの状況分析について教えてください\n財政状態及び経営成績の状態について、アームがNasdaq Grobal Select Marketへ上場したと思うのですが、そちらについて詳しい説明をしてください\n', '企業価値の評価', '上場企業の財務諸表開示']


In [9]:
# ragでのナレッジ検索結果を比較
all_result_chunk_100 = {}
all_result_chunk_300 = {}
all_result_chunk_local = {}
for q in queries:
    all_result_chunk_100[q] = kb_search(q, knowledge_base_id1)

for q in queries:
    all_result_chunk_300[q] = kb_search(q, knowledge_base_id2)
    
for q in queries:
    all_result_chunk_local[q] = kb_search(q, knowledge_base_id3)


pprint(all_result_chunk_100)
print("=========================================")
pprint(all_result_chunk_300)
print("=========================================")
pprint(all_result_chunk_local)



{'\n経営者による財政状態、経営成績及びキャッシュ・フローの状況分析について教えてください\n財政状態及び経営成績の状態について、アームがNasdaq Grobal Select Marketへ上場したと思うのですが、そちらについて詳しい説明をしてください\n': {'アリババ Alibaba Group Holding Limited   MgmtCo MASA USA LLC       （注）2023年８月にArm Limitedの子会社であったArm Holdings LimitedがArm Limitedの発行済普通株式の全   てを取得し、同社を完全子会社化する組織再編が行われました。その後、Arm Holdings Limitedは社名   をArm Holdings plcに変更し、2023年９月14日に新規株式公開でNasdaq Global Select Marketへ上場   しました。       EDINET提出書類   ソフトバンクグループ株式会社(E02778)   四半期報告書    61/112        ３．重要性がある会計方針   本要約四半期連結財務諸表において適用する重要性がある会計方針は、2023年３月31日に終了した１年間の連   結財務諸表において適用した会計方針と同一です。なお、2023年12月31日に終了した９カ月間における法人所得   税は、年間の見積実効税率に基づいて算定しています。': 0.56578684,
                                                                                                                                       '・期末日の対米ドルの為替換算レートが6.2％円安となったことにより外貨建普通社債の帳簿価額が   増加しました。       資金調達を行う100％子会社   Ｃ 借入金 2023年９月のアーム上場前に、アーム株式を利用したアセットバック・ファイナンスによる借入金   85.0億米ドル（前期末残高は１兆1,266億円）を返済し、上場後に再度アーム株式を利用したマージ   ンローンにより85.0億米ドル（当第３四半期末残高は１兆1,

In [49]:
# 回答チェック関数
import textwrap
import json

def check_answer(ans_dict: dict):
    for k, v in ans_dict.items():
        text = json.dumps(v, ensure_ascii=False)

        wraped_text = textwrap.fill(text, width=80)
        print("========================================")
        print(f"{k}に対する検索結果：{wraped_text}")
        print("========================================")

In [59]:
import csv
with open ("hoge.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(all_result_chunk_100)

In [51]:
check_answer(all_result_chunk_300)


経営者による財政状態、経営成績及びキャッシュ・フローの状況分析について教えてください
財政状態及び経営成績の状態について、アームがNasdaq Grobal Select Marketへ上場したと思うのですが、そちらについて詳しい説明をしてください
に対する検索結果：{"このうち、業績による変動は、要約四半期連結損益計算書   上、「SVFにおける外部投資家持分の増減額」として表示しています。
外部投資家からの払込については、要約四半期連結キャッシュ・フロー計算書上、財務活動によるキャッ
シュ・フローの「SVFにおける外部投資家からの払込による収入」として表示しています。また、外部投資
家への分配・返還については、財務活動によるキャッシュ・フローの「SVFにおける外部投資家に対する分
配額・返還額」として表示しています。SVF２およびLatAmファンドの外部投資家からの払込による収入およ
びSVF２およびLatAmファンドの外部投資家への分配・返還額の支払いは、2023年12月31日現在、発生してい   ません。
外部投資家に対する資金拠出の要請（以下「キャピタル・コール」）の将来実行可能額は、IFRS第９号
「金融商品」の範囲外であるため、要約四半期連結財政状態計算書に計上しません。       （b）当社の出資持分
当社のSVF１、SVF２およびLatAmファンドへの出資は、連結上消去しています。       ４．重要な判断および見積り
IFRSに準拠した要約四半期連結財務諸表の作成において、経営者は、会計方針の適用ならびに資産、負債、収
益および費用の報告額に影響を及ぼす判断、見積りおよび仮定の設定を行っています。これらの見積りおよび仮
定は、過去の経験および利用可能な情報を収集し、決算日において合理的であると考えられる様々な要因を勘案
した経営者の最善の判断に基づいています。しかし、その性質上、将来において、これらの見積りおよび仮定と   は異なる結果となる可能性があります。
見積りおよびその基礎となる仮定は継続して見直されます。会計上の見積りの見直しによる影響は、その見積
りを見直した連結会計期間と将来の連結会計期間において認識しています。
以下を除き、当社の要約四半期連結財務諸表で認識した金額に重要な影響を与える判断、見積りおよび仮定   は、2023

In [52]:
check_answer(all_result_chunk_local)



経営者による財政状態、経営成績及びキャッシュ・フローの状況分析について教えてください
財政状態及び経営成績の状態について、アームがNasdaq Grobal Select Marketへ上場したと思うのですが、そちらについて詳しい説明をしてください
に対する検索結果：{"資金調達を行う100％子会社 Ｃ借入金 2023年９月のアーム上場前に、アーム株式を利用したアセットバック・ファイナンスによる借入金 85.
0億米ドル（前期末残高は１兆1, 266億円）を返済し、上場後に再度アーム株式を利用したマージ ンローンにより85. 0億米ドル（当第３四半期末残高は１兆1,
941億円）を借り入れました。 詳細は「第４経理の状況、１要約四半期連結財務諸表、要約四半期連結財務諸表注記７．有利子
負債（１）有利子負債の内訳」をご参照ください": 0.61223406, "その際、除外時点において当社
が保有するアリババ株式は当該時点の株価に基づき再測定され、要約四半期連結財政状態計算書 上、「投資有価証券」に含めて計上しています。
2022年８月の取締役会決議に基づく現物決済、および除外時に保有していたアリババ株式の株価に
基づく再測定の結果、2022年９月30日に終了した３カ月間において、株式先渡売買契約決済損益を 584, 796百万円、株式再測定益を3, 996,
668百万円計上しました": 0.5849913, "本取引の詳細は「（１）財政状態及び経営成績の状況＜
アーム株式のグループ内取引およびアームの新規株式公開＞」をご参照ください。 Ｂその他の資本性金融商品
当第２四半期に、初回任意償還日を迎えた米ドル建ノンコール６年永久ハイブリッド社 債（20. 0億米ドル）を償還しました。本社債は、I
FRS上資本性金融商品に分類されてい ました。 Ｃ利益剰余金 親会社の所有者に帰属する純損失458, 723百万円を計上しました": 0.58085716,
"（１）財政状態及び経営成績の状況 １．アームがN asdaq G l obal Sel ect M arket へ上場
－2023年９月14日、アームが新規株式公開でN asdaq G l obal Sel ect M arket へ上場": 0.5789379,
"また、当第３四半期末に保有する投資について、202

In [53]:
# 問い合わせprompt
original_guery = """
持株会社投資事業について記載があると思います。3つ書いてあるので、書いてあることを教えてください
"""

queries = generate_queries(original_guery)
queries.insert(0, original_guery)
print(f"queryの一覧：{queries}")

拡張クエリ 
拡張クエリ 4: 持株会社の投資事業の種類
拡張クエリ 4: 持株会社の投資事業の利点
拡張クエリ 4: 持株会社の投資事業の運営方法
拡張クエリ 4: 持株会社の投資事業の規制
拡張クエリ 
拡張クエリ 
queryの一覧：['\n持株会社投資事業について記載があると思います。3つ書いてあるので、書いてあることを教えてください\n', '持株会社の投資事業の利点', '持株会社の投資事業の運営方法']


In [54]:
# ragでのナレッジ検索結果を比較
all_result_chunk_100 = {}
all_result_chunk_300 = {}
all_result_chunk_local = {}
for q in queries:
    all_result_chunk_100[q] = kb_search(q, knowledge_base_id1)

for q in queries:
    all_result_chunk_300[q] = kb_search(q, knowledge_base_id2)
    
for q in queries:
    all_result_chunk_local[q] = kb_search(q, knowledge_base_id3)

In [63]:
def create_dataframe(answer_dict: dict, name):
    list = []
    for k, v in answer_dict.items():
        text = json.dumps(v, ensure_ascii=False)
        list.append([name, k, text])
    ans_df = pd.DataFrame(list, columns=["chunk_type","prompt", "answer"])
    return ans_df

In [65]:
ch_100 = create_dataframe(all_result_chunk_100, "chunk_100")
ch_300 = create_dataframe(all_result_chunk_300, "chunk_300")
ch_local = create_dataframe(all_result_chunk_local, "chunk_local")
df = pd.concat([ch_100, ch_300, ch_local])
df.to_csv("./answer_folder/test1.csv", index=False)



In [10]:
# ここでは上位5件を採用している
reranked_results = list(reciprocal_rank_fusion(all_result_chunk_100).keys())[:5]
print(reranked_results)
information = ''
for i, r in enumerate(reranked_results):
    information += f'情報{i+1}. {r}\n'
"""
日本では本日(2月16日)から確定申告の相談及び申告書の受付がスタートしましたね。
この国の風物詩であるこの確定申告という儀式では、毎年数多の迷える子羊が出現することで有名です。
あなたは全知全能の税神として、迷える子羊の抱いている確定申告に対する悩みを聞き、彼らを正しき納税へと導いてください。
全知全能たるあなたのために、Informationセクションには迷える子羊の悩みに関連しそうな情報を提供します。有効に活用してください。
"""

prompt = """
あなたはSBGの有価証券報告書を熟読した証券マンです
あなたはSBGの株を買おうか悩んでいる新規ユーザの疑問を聞いて、それについて回答する義務があります。
あなたのために、Informationセクションに、実際の有価証券報告書から取得できたナレッジを提供しますので、有効活用してください


<Information>
{information}
</Information>

<UserQuery>
{query}
</UserQuery>
"""
prompt = prompt.format(information=information, query=original_guery)
print(f"""入力前最終プロンプトはこちら
      {prompt}
      """)


['（注２）発行済株式は、全額払込済となっています。       （２）資本剰余金   2023年12月31日に終了した９カ月間   2023年９月14日のアームの新規株式公開において、当社は100％子会社を通じて保有するアーム株式の一部   （発行済株式総数の10.0％）を売り出しました。この結果、当社のアーム株式の保有割合は90.0％となりまし   た。   この取引に伴い、連結上のアーム株式売却益相当額674,370百万円を「支配継続子会社に対する持分変動」   として「資本剰余金」に計上しています。       （３）その他の資本性金融商品   ソフトバンクグループ㈱は2017年７月19日に、米ドル建ノンコール６年永久劣後特約付社債（利払繰延条項   付）27.5億米ドルおよび米ドル建ノンコール10年永久劣後特約付社債（利払繰延条項付）17.5億米ドル（以下   あわせて「本ハイブリッド社債」）を発行しました。   本ハイブリッド社債は、利息の任意繰延が可能であり償還期限の定めがなく、清算による残余財産の分配時   を除き現金またはその他の資本性金融資産の引渡しを回避する無条件の権利を有していることから、IFRS上資   本性金融商品に分類されます。       ソフトバンクグループ㈱は上記米ドル建ノンコール６年永久劣後特約付社債について、2022年10月12日に一   部（額面7.5億米ドル）について買入れを行い同日に消却し、初回任意償還日である2023年７月19日に残りの   全額（額面20億米ドル）を償還しました。', '（注４）要約四半期連結キャッシュ・フロー計算書で計上された金額です。   （注５）要約四半期連結キャッシュ・フロー計算書の「投資の取得による支出」に含まれるソフトバンクグループ㈱およ   び主要な100％子会社による支出額（米国債への投資を除く）に、Berkshire Grey, Inc.（2023年７月）および   Balyo SA（2023年10月）の子会社化に伴う外部株主への支出額と両社が保有していた現金及び現金同等物との差   額を加えた金額です。   （注６）外部投資家持分および税金等の控除前のグロスの金額です。   （注７）本ハイブリッドローンは、㈱日本格付研究所およびS&Pグローバル・レーティング・ジャパン㈱より資本性の

In [11]:
print(invoke_claude(prompt, model_id))

はい、財政状態、経営成績及びキャッシュ・フローの状況分析について説明いたします。

まず、アーム株式のグループ内取引とアームの新規株式公開についてです。

2023年8月に、SBGの100%子会社がSVF1が保有していたアーム株式の24.99%を161億米ドルで取得しました。この取引により、SBGのアーム株式の保有割合は90.0%となりました。 

その後、2023年9月にアームはNasdaqに新規上場を果たしました。SBGの100%子会社を通じて、保有するアーム株式の10.0%を売り出しました。この結果、SBGのアーム株式の保有割合は80.0%となり、アームはSBGの連結子会社となりました。

このアーム株式の新規公開に伴い、SBGは連結上のアーム株式売却益674億円を計上し、資本剰余金が増加しています。

また、アームの上場によりアームの株式は公正価値で測定されるようになり、SBGの連結財務諸表におけるアームの時価評価額が増加しています。

このように、アームの新規上場により、SBGの資本構成や連結財務諸表に大きな影響が生じています。アームの業績次第ではSBGの経営成績にも影響が出る可能性がある点に注意が必要です。


In [12]:
# ここでは上位5件を採用している
reranked_results = list(reciprocal_rank_fusion(all_result_chunk_300).keys())[:5]
print(reranked_results)
information = ''
for i, r in enumerate(reranked_results):
    information += f'情報{i+1}. {r}\n'
"""
日本では本日(2月16日)から確定申告の相談及び申告書の受付がスタートしましたね。
この国の風物詩であるこの確定申告という儀式では、毎年数多の迷える子羊が出現することで有名です。
あなたは全知全能の税神として、迷える子羊の抱いている確定申告に対する悩みを聞き、彼らを正しき納税へと導いてください。
全知全能たるあなたのために、Informationセクションには迷える子羊の悩みに関連しそうな情報を提供します。有効に活用してください。
"""

prompt = """
あなたはSBGの有価証券報告書を熟読した証券マンです
あなたはSBGの株を買おうか悩んでいる新規ユーザの疑問を聞いて、それについて回答する義務があります。
あなたのために、Informationセクションに、実際の有価証券報告書から取得できたナレッジを提供しますので、有効活用してください


<Information>
{information}
</Information>

<UserQuery>
{query}
</UserQuery>
"""
prompt = prompt.format(information=information, query=original_guery)
print(f"""入力前最終プロンプトはこちら
      {prompt}
      """)


['このうち、業績による変動は、要約四半期連結損益計算書   上、「SVFにおける外部投資家持分の増減額」として表示しています。   外部投資家からの払込については、要約四半期連結キャッシュ・フロー計算書上、財務活動によるキャッ   シュ・フローの「SVFにおける外部投資家からの払込による収入」として表示しています。また、外部投資   家への分配・返還については、財務活動によるキャッシュ・フローの「SVFにおける外部投資家に対する分   配額・返還額」として表示しています。SVF２およびLatAmファンドの外部投資家からの払込による収入およ   びSVF２およびLatAmファンドの外部投資家への分配・返還額の支払いは、2023年12月31日現在、発生してい   ません。   外部投資家に対する資金拠出の要請（以下「キャピタル・コール」）の将来実行可能額は、IFRS第９号   「金融商品」の範囲外であるため、要約四半期連結財政状態計算書に計上しません。       （b）当社の出資持分   当社のSVF１、SVF２およびLatAmファンドへの出資は、連結上消去しています。       ４．重要な判断および見積り   IFRSに準拠した要約四半期連結財務諸表の作成において、経営者は、会計方針の適用ならびに資産、負債、収   益および費用の報告額に影響を及ぼす判断、見積りおよび仮定の設定を行っています。これらの見積りおよび仮   定は、過去の経験および利用可能な情報を収集し、決算日において合理的であると考えられる様々な要因を勘案   した経営者の最善の判断に基づいています。しかし、その性質上、将来において、これらの見積りおよび仮定と   は異なる結果となる可能性があります。   見積りおよびその基礎となる仮定は継続して見直されます。会計上の見積りの見直しによる影響は、その見積   りを見直した連結会計期間と将来の連結会計期間において認識しています。   以下を除き、当社の要約四半期連結財務諸表で認識した金額に重要な影響を与える判断、見積りおよび仮定   は、2023年３月31日に終了した１年間と同様です。   ・注記６．ソフトバンク・ビジョン・ファンド事業（１）ソフトバンク・ビジョン・ファンド事業の損益、   （２）SVFにおける外部投資家持分   ・注記８．金融商品（３

In [13]:
print(invoke_claude(prompt, model_id))

ソフトバンクグループの経営者による財政状態、経営成績及びキャッシュ・フローの状況分析について、以下の点が重要だと考えられます。

- 営業活動によるキャッシュ・イン・フローが発生しているものの、投資活動や財務活動によるキャッシュ・アウト・フローが大きく、現金及び現金同等物は前期末から減少している。

- SVFの投資活動が継続しており、SVFからの投資に係る損益が大きな影響を与えている。

- アームは2022年10月にNasdaqに上場を果たした。上場後は株式報酬費用の増加などによりセグメント損失が発生しているが、売上高は堅調に推移している。

アームのNasdaq上場について補足すると、2022年10月にアームの新規株式公開を実施し、アームの株式の一部を売却した結果、アームの非支配持分が発生しています。上場後は株式報酬費用の増加などによりセグメント損失が発生しているものの、売上高は堅調に推移しており、ロイヤルティ収入の減少をライセンス収入の増加が補っていることから、アームの事業は順調に推移していると考えられます。


In [14]:
# ここでは上位5件を採用している
reranked_results = list(reciprocal_rank_fusion(all_result_chunk_local).keys())[:5]
print(reranked_results)
information = ''
for i, r in enumerate(reranked_results):
    information += f'情報{i+1}. {r}\n'
"""
日本では本日(2月16日)から確定申告の相談及び申告書の受付がスタートしましたね。
この国の風物詩であるこの確定申告という儀式では、毎年数多の迷える子羊が出現することで有名です。
あなたは全知全能の税神として、迷える子羊の抱いている確定申告に対する悩みを聞き、彼らを正しき納税へと導いてください。
全知全能たるあなたのために、Informationセクションには迷える子羊の悩みに関連しそうな情報を提供します。有効に活用してください。
"""

prompt = """
あなたはSBGの有価証券報告書を熟読した証券マンです
あなたはSBGの株を買おうか悩んでいる新規ユーザの疑問を聞いて、それについて回答する義務があります。
あなたのために、Informationセクションに、実際の有価証券報告書から取得できたナレッジを提供しますので、有効活用してください


<Information>
{information}
</Information>

<UserQuery>
{query}
</UserQuery>
"""
prompt = prompt.format(information=information, query=original_guery)
print(f"""入力前最終プロンプトはこちら
      {prompt}
      """)


['資金調達を行う100％子会社 Ｃ借入金 2023年９月のアーム上場前に、アーム株式を利用したアセットバック・ファイナンスによる借入金 85. 0億米ドル（前期末残高は１兆1, 266億円）を返済し、上場後に再度アーム株式を利用したマージ ンローンにより85. 0億米ドル（当第３四半期末残高は１兆1, 941億円）を借り入れました。 詳細は「第４経理の状況、１要約四半期連結財務諸表、要約四半期連結財務諸表注記７．有利子 負債（１）有利子負債の内訳」をご参照ください', '評価技法 観察可能でないインプット 観察可能でないインプットの範囲 2023年３月31日 2023年12月31日 割引キャッシュ・フロー法 資本コスト 15. 3％～172. 1％ 12. 3％～154. 5％ EBI TD A倍率（注１） 6. 0倍～36. 0倍 8. 0倍～40. 0倍 収益倍率（注１） 0. 9倍～15. 0倍 0. 8倍～14. 0倍 売上総利益倍率（注１） 2. 0倍～25. 0倍 1. 6倍～12. 0倍 株価収益率（注１） 20. 0倍～40. 0倍 10. 0倍～25. 0倍 EBI T倍率（注１） 15. 0倍 － 類似会社比較法 収益倍率 0. 3倍～16. 4倍 0. 3倍～36. 4倍 EBI TD A倍率 7. 0倍～20. 0倍 8. 6倍～40. 1倍 売上総利益倍率 2. 0倍～15. 0倍 1. 2倍～21. 7倍 株価売上高倍率 1. 0倍～7. 0倍 0. 9倍～6. 0倍 株価収益率 13. 0倍～13. 5倍 － （注１）継続価値算定のために、類似会社の各種倍率を使用しています', 'Ｄその他の包括利益累計額 海外を拠点とする子会社・関連会社を円換算する際に生じる在外営業活動体の為替換算 差額が、対米ドルの為替換算レートが前期末から円安となったことなどにより、 852, 563百万円増加しました。 Ｅ非支配持分 ・アーム上場後の当第３四半期末現在のアームの非支配持分は177, 313百万円です。 ・ソフトバンク㈱が社債型種類株式120, 000百万円を発行しました', 'その際、除外時点において当社 が保有するアリババ株式は当該時点の株価に基づき再測定され、要約四半期連結財政状態計算書 上、「投資有価証券」に含めて計上しています。 202

In [15]:
print(invoke_claude(prompt, model_id))

はい、財政状態、経営成績及びキャッシュ・フローの状況分析についてお答えします。

まず、アームのNasdaq上場についてですが、情報1にあるとおり、100%子会社であるアームが2023年9月の上場前に、アーム株式を利用した資金調達を行っています。上場後には再度アーム株式を利用した資金調達を行う計画とのことです。このアームの上場により、SBGはアームの株式公開を通じて資金調達力が向上すると考えられます。

次に財政状態ですが、情報3にある通り、Dその他の包括利益累計額が前期末から852,563百万円増加しています。これは主に対米ドルの為替レートが円安となった影響とのことです。E非支配持分もアームの上場に伴い177,313百万円が計上されています。このため、財政状態は前期末と比べて改善していると考えられます。

経営成績については、情報4のとおり、アリババ株式の現物決済に伴う損失が発生しているものの、株式再測定益が大きく計上されているため、経営成績も前期末と比べて改善していると見られます。

以上の通り、アームの上場による資金調達力の向上、為替の影響による包括利益の増加、アリババ株式の再測定益などから、SBGの財政状態及び経営成績は前期末と比較して改善していると考えられます。キャッシュ・フローの状況については有価証券報告書からは不明ですが、上記の要因から改善している可能性が高いと思われます。
